In [1]:
# Cell 1: Setup and Imports
import sys
import time
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Force reload of modules
for mod in list(sys.modules.keys()):
    if 'qectostim' in mod:
        del sys.modules[mod]

import stim
from qectostim.codes import discover_all_codes
from qectostim.experiments.memory import CSSMemoryExperiment
from qectostim.noise.models import CircuitDepolarizingNoise

# Discover all CSS codes
discovered = discover_all_codes(max_qubits=50, include_qldpc=False)
codes = {name: code for name, code in discovered.items() 
         if hasattr(code, 'hx') and hasattr(code, 'hz')}

# Filter to distance >= 3 (decoders work on error correction, not detection)
codes = {name: code for name, code in codes.items() 
         if isinstance(code.metadata.get('distance'), int) and code.metadata.get('distance') >= 3}

print(f"Discovered {len(codes)} CSS codes with d≥3 for decoder testing")
print()
for name, code in codes.items():
    d = code.metadata.get('distance', '?')
    print(f"  {name}: [[{code.n},{code.k},{d}]]")

Discovered 16 CSS codes with d≥3 for decoder testing

  Steane_713: [[7,1,3]]
  Shor_91: [[9,1,3]]
  ReedMuller_15_1_3: [[15,1,3]]
  ToricCode_3x3: [[18,2,3]]
  Hamming_CSS_7: [[7,1,3]]
  RotatedSurface_[[9,1,3]]: [[9,1,3]]
  RotatedSurface_[[25,1,5]]: [[25,1,5]]
  TriangularColour_d3: [[7,1,3]]
  HexagonalColour_d3: [[17,7,3]]
  XZZX_Surface_3: [[9,1,3]]
  XZZX_Surface_5: [[25,1,5]]
  GaugeColor_3: [[22,18,3]]
  Repetition_3: [[3,1,3]]
  Repetition_5: [[5,1,5]]
  Repetition_7: [[7,1,7]]
  Repetition_9: [[9,1,9]]


In [2]:
# Cell 2: Load Available Decoders

decoder_classes = {}

# PyMatching
try:
    from qectostim.decoders.pymatching_decoder import PyMatchingDecoder
    decoder_classes['PyMatching'] = PyMatchingDecoder
except: pass

# Fusion Blossom
try:
    from qectostim.decoders.fusion_blossom_decoder import FusionBlossomDecoder
    decoder_classes['FusionBlossom'] = FusionBlossomDecoder
except: pass

# Belief Matching
try:
    from qectostim.decoders.belief_matching import BeliefMatchingDecoder
    decoder_classes['BeliefMatching'] = BeliefMatchingDecoder
except: pass

# BP-OSD
try:
    from qectostim.decoders.bp_osd import BPOSDDecoder
    decoder_classes['BPOSD'] = BPOSDDecoder
except: pass

# Tesseract
try:
    from qectostim.decoders.tesseract_decoder import TesseractDecoder
    decoder_classes['Tesseract'] = TesseractDecoder
except: pass

# Union Find (fallback to PyMatching)
try:
    from qectostim.decoders.union_find_decoder import UnionFindDecoder
    decoder_classes['UnionFind'] = UnionFindDecoder
except: pass

print(f"Loaded {len(decoder_classes)} decoders: {list(decoder_classes.keys())}")

Loaded 6 decoders: ['PyMatching', 'FusionBlossom', 'BeliefMatching', 'BPOSD', 'Tesseract', 'UnionFind']


In [3]:
# Cell 3: Test Helper Function

def test_decoder_on_code(code, decoder_name, decoder_class, p=0.01, shots=1000, rounds=3):
    """Test a decoder on a code and return results dict."""
    result = {
        'status': 'UNKNOWN',
        'ler': None,
        'ler_no_decode': None,
        'time_ms': None,
        'warnings': []
    }
    
    try:
        noise = CircuitDepolarizingNoise(p1=p, p2=p)
        exp = CSSMemoryExperiment(code=code, rounds=rounds, noise_model=noise)
        circuit = noise.apply(exp.to_stim())
        
        try:
            dem = circuit.detector_error_model(decompose_errors=True)
        except:
            dem = circuit.detector_error_model(decompose_errors=True, ignore_decomposition_failures=True)
        
        # Sample
        sampler = dem.compile_sampler()
        raw = sampler.sample(shots, bit_packed=False)
        
        if isinstance(raw, tuple):
            det_samples = np.asarray(raw[0], dtype=np.uint8)
            obs_samples = np.asarray(raw[1], dtype=np.uint8)
        else:
            arr = np.asarray(raw, dtype=np.uint8)
            det_samples = arr[:, :dem.num_detectors]
            obs_samples = arr[:, dem.num_detectors:]
        
        if obs_samples.shape[1] > 0:
            result['ler_no_decode'] = float(obs_samples[:, 0].mean())
        
        # Create decoder
        if decoder_name in ['PyMatching', 'FusionBlossom']:
            decoder = decoder_class(dem)
        else:
            decoder = decoder_class(dem=dem)
        
        # Decode
        start = time.time()
        corrections = decoder.decode_batch(det_samples)
        result['time_ms'] = (time.time() - start) * 1000
        
        corrections = np.asarray(corrections, dtype=np.uint8)
        if corrections.ndim == 1:
            corrections = corrections.reshape(-1, max(1, dem.num_observables))
        
        if obs_samples.shape[1] > 0:
            logical_errors = (corrections[:, 0] ^ obs_samples[:, 0]).astype(np.uint8)
            result['ler'] = float(logical_errors.mean())
        
        result['status'] = 'OK'
        
        # Warnings
        ler, ler_nd = result['ler'], result['ler_no_decode']
        if ler is not None:
            if ler < 1e-6 and p > 0.001:
                result['warnings'].append('LER≈0')
            if ler_nd is not None and ler >= ler_nd and code.metadata.get('distance', 0) >= 3:
                result['warnings'].append('No improvement')
        
    except Exception as e:
        result['status'] = 'FAIL'
        result['error'] = str(e)[:50]
    
    return result

print("✓ Test helper defined")

✓ Test helper defined


In [4]:
# Cell 4: DECODER × CODE COMPATIBILITY MATRIX

print("="*100)
print("DECODER × CODE COMPATIBILITY MATRIX")
print("="*100)
print("\nTesting all decoder/code combinations at p=0.01, 1000 shots...\n")

p = 0.01
shots = 1000

# Store results
full_results = {}

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<25}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:^12}"
print(header)
print("-" * len(header))

for code_name, code in codes.items():
    row = f"{code_name:<25}"
    full_results[code_name] = {}
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=code,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=shots
        )
        full_results[code_name][dec_name] = result
        
        if result['status'] == 'OK':
            ler = result['ler']
            if result['warnings']:
                cell = f"⚠️{ler:.4f}"
            else:
                cell = f"✓ {ler:.4f}"
        else:
            cell = f"✗ FAIL"
        
        row += f" | {cell:^12}"
    
    print(row)

print("-" * len(header))

DECODER × CODE COMPATIBILITY MATRIX

Testing all decoder/code combinations at p=0.01, 1000 shots...

Code                      |  PyMatching  |  FusionBlos  |  BeliefMatc  |    BPOSD     |  Tesseract   |  UnionFind  
-------------------------------------------------------------------------------------------------------------------
Steane_713                |   ⚠️0.0000   |   ⚠️0.0000   |   ⚠️0.0000   |    ✗ FAIL    |   ⚠️0.0000   |   ⚠️0.0000  
Steane_713                |   ⚠️0.0000   |   ⚠️0.0000   |   ⚠️0.0000   |    ✗ FAIL    |   ⚠️0.0000   |   ⚠️0.0000  
Shor_91                   |   ✓ 0.0040   |   ✓ 0.0030   |   ✓ 0.0060   |   ✓ 0.0040   |   ✓ 0.0020   |   ✓ 0.0010  
Shor_91                   |   ✓ 0.0040   |   ✓ 0.0030   |   ✓ 0.0060   |   ✓ 0.0040   |   ✓ 0.0020   |   ✓ 0.0010  
ReedMuller_15_1_3         |   ✓ 0.0790   |   ✓ 0.0840   |   ✓ 0.0440   |   ✓ 0.0350   |   ✓ 0.0250   |   ✓ 0.0790  
ReedMuller_15_1_3         |   ✓ 0.0790   |   ✓ 0.0840   |   ✓ 0.0440   |   ✓ 0.0350   |

In [5]:
# Cell 5: LER COMPARISON TABLE

print("="*100)
print("LER COMPARISON TABLE (p=0.01)")
print("="*100)
print("\nLower is better. Best decoder for each code highlighted.\n")

# Header
dec_names = list(decoder_classes.keys())
header = f"{'Code':<25} | {'d':>2} | {'No-decode':>10}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
header += " | Best"
print(header)
print("-" * len(header))

for code_name, code in codes.items():
    d = code.metadata.get('distance', '?')
    code_results = full_results.get(code_name, {})
    
    # Get no-decode LER
    ler_no_decode = None
    for res in code_results.values():
        if res.get('ler_no_decode') is not None:
            ler_no_decode = res['ler_no_decode']
            break
    
    nd_str = f"{ler_no_decode:.4f}" if ler_no_decode else 'N/A'
    row = f"{code_name:<25} | {d:>2} | {nd_str:>10}"
    
    # Find best decoder
    best_ler = float('inf')
    best_decoder = None
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        ler = res.get('ler')
        if ler is not None:
            ler_str = f"{ler:.4f}"
            if ler < best_ler:
                best_ler = ler
                best_decoder = dec_name
        else:
            ler_str = 'FAIL'
        row += f" | {ler_str:>10}"
    
    best_str = best_decoder[:10] if best_decoder else 'N/A'
    row += f" | {best_str}"
    print(row)

print("-" * len(header))

LER COMPARISON TABLE (p=0.01)

Lower is better. Best decoder for each code highlighted.

Code                      |  d |  No-decode | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind | Best
--------------------------------------------------------------------------------------------------------------------------------
Steane_713                |  3 |        N/A |     0.0000 |     0.0000 |     0.0000 |       FAIL |     0.0000 |     0.0000 | PyMatching
Shor_91                   |  3 |     0.2110 |     0.0040 |     0.0030 |     0.0060 |     0.0040 |     0.0020 |     0.0010 | UnionFind
ReedMuller_15_1_3         |  3 |     0.1490 |     0.0790 |     0.0840 |     0.0440 |     0.0350 |     0.0250 |     0.0790 | Tesseract
ToricCode_3x3             |  3 |     0.1190 |     0.0050 |     0.0090 |     0.0160 |     0.0080 |     0.0080 |     0.0080 | PyMatching
Hamming_CSS_7             |  3 |     0.0990 |     0.0920 |     0.0950 |     0.0660 |     0.0610 |     0.0190 |     

In [6]:
# Cell 6: DECODER SPEED COMPARISON

print("="*100)
print("DECODER SPEED COMPARISON (time in ms for 1000 shots)")
print("="*100)

# Header
dec_names = list(decoder_classes.keys())
header = f"\n{'Code':<25}"
for dec_name in dec_names:
    header += f" | {dec_name[:10]:>10}"
print(header)
print("-" * len(header))

for code_name in codes.keys():
    code_results = full_results.get(code_name, {})
    row = f"{code_name:<25}"
    
    for dec_name in dec_names:
        res = code_results.get(dec_name, {})
        time_ms = res.get('time_ms')
        time_str = f"{time_ms:.1f}" if time_ms else 'FAIL'
        row += f" | {time_str:>10}"
    
    print(row)

print("-" * len(header))

DECODER SPEED COMPARISON (time in ms for 1000 shots)

Code                      | PyMatching | FusionBlos | BeliefMatc |      BPOSD |  Tesseract |  UnionFind
--------------------------------------------------------------------------------------------------------
Steane_713                |        0.1 |        3.3 |        4.4 |       FAIL |        8.5 |        0.1
Shor_91                   |        0.2 |        4.4 |        9.1 |       12.6 |       10.7 |        0.2
ReedMuller_15_1_3         |        0.7 |       11.4 |      115.6 |      362.3 |       27.1 |        0.6
ToricCode_3x3             |        0.8 |       14.1 |       63.2 |      127.5 |       27.3 |        0.7
Hamming_CSS_7             |        0.3 |        5.1 |       17.4 |       38.8 |       30.9 |        0.3
RotatedSurface_[[9,1,3]]  |        0.7 |        6.0 |       14.6 |       35.3 |       13.0 |        0.3
RotatedSurface_[[25,1,5]] |        1.1 |       17.8 |      108.8 |      239.9 |       26.9 |        1.0
Triangula

In [7]:
# Cell 7: NOISE LEVEL SCALING TEST

print("="*100)
print("NOISE LEVEL SCALING TEST")
print("="*100)

# Pick a representative code (RotatedSurface d=3)
test_code_name = None
for name, code in codes.items():
    if 'RotatedSurface' in name and code.metadata.get('distance') == 3:
        test_code_name = name
        break

if test_code_name is None:
    test_code_name = list(codes.keys())[0]  # Fallback to first code

test_code = codes[test_code_name]
print(f"\nTesting {test_code_name} across noise levels with all decoders...\n")

noise_levels = [0.001, 0.005, 0.01, 0.02]

# Header
dec_names = list(decoder_classes.keys())
header = f"{'p':<10}"
for dec_name in dec_names:
    header += f" | {dec_name[:12]:>12}"
header += " | No-decode"
print(header)
print("-" * len(header))

for p in noise_levels:
    row = f"{p:<10.4f}"
    ler_no_decode = None
    
    for dec_name in dec_names:
        result = test_decoder_on_code(
            code=test_code,
            decoder_name=dec_name,
            decoder_class=decoder_classes[dec_name],
            p=p,
            shots=2000,
            rounds=3
        )
        
        if ler_no_decode is None and result.get('ler_no_decode'):
            ler_no_decode = result['ler_no_decode']
        
        ler = result.get('ler')
        ler_str = f"{ler:.6f}" if ler else 'FAIL'
        row += f" | {ler_str:>12}"
    
    nd_str = f"{ler_no_decode:.6f}" if ler_no_decode else 'N/A'
    row += f" | {nd_str}"
    print(row)

print("-" * len(header))

NOISE LEVEL SCALING TEST

Testing RotatedSurface_[[9,1,3]] across noise levels with all decoders...

p          |   PyMatching | FusionBlosso | BeliefMatchi |        BPOSD |    Tesseract |    UnionFind | No-decode
----------------------------------------------------------------------------------------------------------------
0.0010     |         FAIL |         FAIL |     0.000500 |         FAIL |         FAIL |     0.000500 | 0.008500
0.0010     |         FAIL |         FAIL |     0.000500 |         FAIL |         FAIL |     0.000500 | 0.008500
0.0050     |     0.005500 |     0.002500 |     0.007500 |     0.004500 |     0.005500 |     0.004000 | 0.048000
0.0050     |     0.005500 |     0.002500 |     0.007500 |     0.004500 |     0.005500 |     0.004000 | 0.048000
0.0100     |     0.015500 |     0.014500 |     0.018000 |     0.015500 |     0.018000 |     0.021000 | 0.109500
0.0100     |     0.015500 |     0.014500 |     0.018000 |     0.015500 |     0.018000 |     0.021000 | 0.109500
0

In [8]:
# Cell 8: FINAL SUMMARY

print("="*100)
print("FINAL DECODER SMOKE TEST SUMMARY")
print("="*100)

# Count statistics
total_tests = 0
passed = 0
failed = 0
with_warnings = 0

for code_name, code_results in full_results.items():
    for dec_name, result in code_results.items():
        total_tests += 1
        if result['status'] == 'OK':
            if result['warnings']:
                with_warnings += 1
            else:
                passed += 1
        else:
            failed += 1

print(f"\n📊 RESULTS:")
print(f"   Total decoder × code tests: {total_tests}")
print(f"   ✓ Passed:   {passed} ({100*passed/total_tests:.1f}%)")
print(f"   ⚠ Warnings: {with_warnings} ({100*with_warnings/total_tests:.1f}%)")
print(f"   ✗ Failed:   {failed} ({100*failed/total_tests:.1f}%)")

print(f"\n🔧 DECODERS TESTED: {len(decoder_classes)}")
for name in decoder_classes.keys():
    print(f"   - {name}")

print(f"\n📦 CODES TESTED: {len(codes)}")
for name, code in codes.items():
    d = code.metadata.get('distance', '?')
    print(f"   - {name}: [[{code.n},{code.k},{d}]]")

if failed > 0:
    print(f"\n⚠ FAILURES:")
    for code_name, code_results in full_results.items():
        for dec_name, result in code_results.items():
            if result['status'] != 'OK':
                print(f"   - {dec_name} + {code_name}: {result.get('error', 'unknown')[:40]}")

print("\n" + "="*100)
if failed == 0:
    print("✓ ALL TESTS PASSED")
else:
    print(f"⚠ {failed} TESTS FAILED - see details above")
print("="*100)

FINAL DECODER SMOKE TEST SUMMARY

📊 RESULTS:
   Total decoder × code tests: 96
   ✓ Passed:   53 (55.2%)
   ⚠ Warnings: 40 (41.7%)
   ✗ Failed:   3 (3.1%)

🔧 DECODERS TESTED: 6
   - PyMatching
   - FusionBlossom
   - BeliefMatching
   - BPOSD
   - Tesseract
   - UnionFind

📦 CODES TESTED: 16
   - Steane_713: [[7,1,3]]
   - Shor_91: [[9,1,3]]
   - ReedMuller_15_1_3: [[15,1,3]]
   - ToricCode_3x3: [[18,2,3]]
   - Hamming_CSS_7: [[7,1,3]]
   - RotatedSurface_[[9,1,3]]: [[9,1,3]]
   - RotatedSurface_[[25,1,5]]: [[25,1,5]]
   - TriangularColour_d3: [[7,1,3]]
   - HexagonalColour_d3: [[17,7,3]]
   - XZZX_Surface_3: [[9,1,3]]
   - XZZX_Surface_5: [[25,1,5]]
   - GaugeColor_3: [[22,18,3]]
   - Repetition_3: [[3,1,3]]
   - Repetition_5: [[5,1,5]]
   - Repetition_7: [[7,1,7]]
   - Repetition_9: [[9,1,9]]

⚠ FAILURES:
   - BPOSD + Steane_713: ERROR: OSD order '-12' invalid. Please c
   - BPOSD + TriangularColour_d3: ERROR: OSD order '-12' invalid. Please c
   - BPOSD + HexagonalColour_d3: ERROR: 